In [1]:
%%time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re

import openpyxl
from sklearn.feature_extraction.text import TfidfVectorizer

Wall time: 4.66 s


In [2]:
%%time
cpuData_morph_KR = pd.read_csv('./NN_KR.csv')
cpuData_morph_ENG = pd.read_csv('./NN_ENG.csv')

Wall time: 1.56 s


In [3]:
%%time
cpuData_csv = pd.read_csv('./cpuDataSet.csv',  encoding = 'CP949')

Wall time: 1.57 s


In [4]:
%%time
for i in range(len(cpuData_csv['대표청구항'])):
    cpuData_csv['대표청구항'][i] = cpuData_csv['대표청구항'][i][9:]
# cpuData_csv['대표청구항']
for i in range(len(cpuData_csv['발명의 명칭'])):
    cpuData_csv['발명의 명칭'][i] = re.sub(r'\([^)]*\)', '', cpuData_csv['발명의 명칭'][i])
cpuData_KR = cpuData_csv[cpuData_csv['국가코드'].isin(['KR']) | cpuData_csv['국가코드'].isin(['JP'])]
cpuData_ENG = cpuData_csv[cpuData_csv['국가코드'].isin(['CN']) | cpuData_csv['국가코드'].isin(['EP']) | cpuData_csv['국가코드'].isin(['US'])]

Wall time: 5.24 s


In [5]:
%%time
cpuData_KR['꼬꼬마'] = cpuData_KR['발명의 명칭'] + ' ' + cpuData_KR['요약'] + cpuData_KR['대표청구항']
# cpuData_csv['꼬꼬마'] = cpuData_csv['발명의 명칭'][:6887] + ' ' + cpuData_csv['요약'][:6887] + cpuData_csv['대표청구항'][:6887]
# cpuData_csv['꼬꼬마'].drop(cpuData_csv['꼬꼬마'].index[6887:]) # 영어부분 삭제


# # cpuData_csv['nltk'] = cpuData_csv['발명의 명칭'][6887:] + ' ' + cpuData_csv['요약'][6887:] + cpuData_csv['대표청구항'][6887:]
# cpuData_csv['nltk'].drop(cpuData_csv['nltk'].index[:6887]) # 한글부분 삭제
cpuData_ENG['nltk'] = cpuData_ENG['발명의 명칭'] + ' ' + cpuData_ENG['요약'] + cpuData_ENG['대표청구항']

Wall time: 46 ms


In [6]:
%%time
IPC_Number = [ipc[0:4] for ipc in cpuData_csv['메인 IPC'].values]
Unique_IPC_Number = list(set(IPC_Number))

Wall time: 1 ms


In [7]:
%%time
IPC_Probability = []
for u in Unique_IPC_Number:
    IPC_Probability.append((u, (IPC_Number.count(u) / len(IPC_Number))))

Wall time: 26 ms


In [8]:
%%time
cpuData_morph_list_KR = []
cpuData_morph_cleaned_list_KR = []
for i in range(len(cpuData_morph_KR)):
    cpuData_morph_list_KR.append(cpuData_morph_KR.iloc[i][1:].tolist())
for i in range(len(cpuData_morph_list_KR)):
    cpuData_morph_cleaned_list_KR.append([x for x in cpuData_morph_list_KR if str(x) != 'nan'])

Wall time: 12min 42s


In [9]:
%%time
cpuData_morph_list_ENG = []
cpuData_morph_cleaned_list_ENG = []
for i in range(len(cpuData_morph_ENG)):
    cpuData_morph_list_ENG.append(cpuData_morph_ENG.iloc[i][1:].tolist())
for i in range(len(cpuData_morph_list_ENG)):
    cpuData_morph_cleaned_list_ENG.append([x for x in cpuData_morph_list_ENG if str(x) != 'nan'])

Wall time: 1h 34min 2s


In [10]:
# %%time

# for ipc, P_c in IPC_Probability:
#     first_term = first_term + (P_c * np.log2(P_c))
# first_term = first_term * -1
# first_term

In [11]:
# %%time
# cpuData_list = []
# for i in range(len(cpuData_morph_cleaned_list_KR)):
#     cpuData_list = cpuData_list + cpuData_morph_cleaned_list_KR[i]
# for i in range(len(cpuData_morph_cleaned_list_ENG)):
#     cpuData_list = cpuData_list + cpuData_morph_cleaned_list_ENG
# cpuData_list

In [12]:
cpuDF_KR = pd.DataFrame()
cpuDF_KR['Data'] = cpuData_KR['꼬꼬마']
cpuDF_KR['IPC'] = cpuData_KR['메인 IPC']

In [13]:
%%time
result_KR = []
for rows, Document in enumerate(cpuData_morph_cleaned_list_KR):
    for t in Document:
        count_t_of_all = str(cpuDF_KR['Data'].values).count(t) # 전체 문서에서 단어 t의 수
        P_t = count_t_of_all / (len(cpuData_morph_cleaned_list_KR) + len(cpuData_morph_cleaned_list_ENG)) # 전체 문서에서 단어 t의 확률
        p_not_t = 1 - P_t # 전체 문서에서 단어 t가 아닌 확률
        sum_of_p_conditional_c_and_t = 0 # 조건부확률 P(c|t)의 합을 담을 공간
        sum_of_p_conditional_c_and_not_t = 0 # 조건부확률 P(c| not t)의 합을 담을 공간
        first_term = 0 # 첫째 항
        for ipc, P_c in IPC_Probability:
            if ipc == cpuDF_KR.iloc[rows,1][0:4]: # IPC가 같으면

                c_and_t = cpuDF_KR.iloc[rows,0].count(t) # 해당하는 IPC에 대한 단어 t의 수
                p_conditional_c_and_t = c_and_t / count_t_of_all # 해당하는 IPC에 대한 문서에서 단어 t의 수 / 전체 문서에서 단어 t의 수
                sum_of_p_conditional_c_and_t += (p_conditional_c_and_t * np.log2(p_conditional_c_and_t)) # 조건부확률의 합 P(c|t) * log(P(c|t))

                c_and_not_t = len(Document) - cpuDF_KR.iloc[rows,0].count(t)   # 문서 전체 단어 수에서 단어 t의 수를 뺌
                p_conditional_c_and_not_t = c_and_not_t / count_t_of_all    # P(c| not t)를 구함
                sum_of_p_conditional_c_and_not_t += (p_conditional_c_and_not_t * np.log2(p_conditional_c_and_not_t)) # P(c| not t) * log(P(c| not t))

            first_term = first_term + (P_c * np.log2(P_c))
        result_KR.append((t, (-1 * first_term) + (P_t * sum_of_p_conditional_c_and_t) + (p_not_t * sum_of_p_conditional_c_and_not_t)))
        

TypeError: must be str, not list

In [20]:
for rows, Document in enumerate(cpuData_morph_cleaned_list_KR):
    print(Document)

11

In [14]:
%%time
result_df_KR = pd.DataFrame.from_records(result_KR)
result_df_KR.to_csv('IG_KR.csv')
result_df_KR.to_excel('IG_KR.xlsx')

Wall time: 27 ms


In [15]:
cpuDF_ENG = pd.DataFrame()
cpuDF_ENG['Data'] = cpuData_ENG['nltk']
cpuDF_ENG['IPC'] = cpuData_ENG['메인 IPC']

In [16]:
%%time
result_ENG = []
for rows, Document in enumerate(cpuData_morph_cleaned_list_ENG):
    for t in Document:
        count_t_of_all = str(cpuDF_ENG['Data'].values).count(t) # 전체 문서에서 단어 t의 수
        P_t = count_t_of_all / (len(cpuData_morph_cleaned_list_KR) + len(cpuData_morph_cleaned_list_ENG)) # 전체 문서에서 단어 t의 확률
        p_not_t = 1 - P_t # 전체 문서에서 단어 t가 아닌 확률
        sum_of_p_conditional_c_and_t = 0 # 조건부확률 P(c|t)의 합을 담을 공간
        sum_of_p_conditional_c_and_not_t = 0 # 조건부확률 P(c| not t)의 합을 담을 공간
        first_term = 0 # 첫째 항
        for ipc, P_c in IPC_Probability:
            if ipc == cpuDF_ENG.iloc[rows,1][0:4]: # IPC가 같으면

                c_and_t = cpuDF_ENG.iloc[rows,0].count(t) # 해당하는 IPC에 대한 단어 t의 수
                p_conditional_c_and_t = c_and_t / count_t_of_all # 해당하는 IPC에 대한 문서에서 단어 t의 수 / 전체 문서에서 단어 t의 수
                sum_of_p_conditional_c_and_t += (p_conditional_c_and_t * np.log2(p_conditional_c_and_t)) # 조건부확률의 합 P(c|t) * log(P(c|t))

                c_and_not_t = len(Document) - cpuDF_ENG.iloc[rows,0].count(t)   # 문서 전체 단어 수에서 단어 t의 수를 뺌
                p_conditional_c_and_not_t = c_and_not_t / count_t_of_all    # P(c| not t)를 구함
                sum_of_p_conditional_c_and_not_t += (p_conditional_c_and_not_t * np.log2(p_conditional_c_and_not_t)) # P(c| not t) * log(P(c| not t))

            first_term = first_term + (P_c * np.log2(P_c))
        result_ENG.append((t, (-1 * first_term) + (P_t * sum_of_p_conditional_c_and_t) + (p_not_t * sum_of_p_conditional_c_and_not_t)))
        

TypeError: must be str, not list

In [17]:
%%time
result_df_ENG = pd.DataFrame.from_records(result_ENG)
result_df_ENG.to_csv('IG_ENG.csv')
result_df_ENG.to_excel('IG_ENG.xlsx')

Wall time: 16 ms
